# Centrocampistas

Vamos a seleccionar a los dos mejores mediocentros según:

    1. Recuperaciones
    2. Porcentaje de pases acertados
    3. Asistencias
    4. Porcentaje de regates acertados 
    5. Goles

In [1]:

import pandas as pd
import numpy as np

#leemos el csv y eliminamos duplicidades

df = pd.read_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/raw/datasetMatchInfo2020-La-Liga.csv')
df = df.drop_duplicates()



In [2]:
'''Creamos un array donde aparezcan los nombres de todos los jugadores que, 
según el dataset, al menos una vez han jugado como centrocampista'''
df_cm = df[df['Pos'] == 'CM']
df_cm_unique = df_cm['Player'].unique()


In [3]:
'''
Creamos un bucle for que lea todos los nombres del array para 
que les asigne la posición CM en todos los partidos.
'''
for i in df_cm_unique:
    if np.any(df['Player'] == i):
        df.loc[(df['Player']== i), 'Pos'] = 'CM'



/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83371/3213891634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'CM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83371/3213891634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Pos'][df['Player']== i] = 'CM'
/var/folders/6g/cf_j0b453g7fk51dd_1wyyzr0000gn/T/ipykernel_83371/3213891634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [4]:
'''
Para segmentar, establecemos que queremos obtener únicamente los partidos
en los que el futbolista haya jugado más de 45 minutos
'''
df_cm = df[df['Pos'] == 'CM']
df_cm =df_cm[df_cm['Min']>= 45]

In [5]:
#Creamos un dataframe con los minutos jugados por jugador
#Segmentamos por los jugadores que hayan jugado, al menos, un total de 520 minutos.
df_minutos = df_cm.groupby('Player')['Min'].sum()
df_minutos.mean()

520.3805970149253

In [6]:
df_minutos_cm = df_minutos.loc[df_minutos>520].sort_values(ascending =False)


In [7]:
#Creamos el nuevo dataframe con los futbolistas que han jugado más de 520 minutos
df_cm= df_cm[df_cm['Player'].isin(df_minutos_cm.keys())]

In [8]:

df_toques= df_cm.groupby('Player')['Performance|Touches'].sum()
df_toques.mean()


504.6515151515151

In [9]:
#como el mínimo de minutos es de 720 (más de 8 partidos), establezco un mínimo
#de 500 toques de balón en total (la media de toques entre todos los mediocentros estudiados) 
df_toques_cm = df_toques.loc[df_toques>= (500)].sort_values(ascending=False)

In [10]:
#aplico el filtro al dataframe
df_cm= df_cm[df_cm['Player'].isin(df_toques_cm.keys())]

In [11]:
'''
Creamos un dataframe con la media de recuperaciones de balón por partido.
Posteriormente los ponderamos entre el total.
'''
df_recov_try= df_cm.groupby('Player')['Performance|Recov'].mean().sort_values(ascending=False)
df_recov = df_recov_try[df_recov_try.keys().isin(df_minutos_cm.keys())]
df_recov = df_recov/df_recov.sum()


In [12]:
'''
Creamos un dataframe con las asistencias realizadas por jugador.
'''
df_ast_try= df_cm.groupby('Player')['Performance|Ast'].sum().sort_values(ascending=False)
df_ast = df_ast_try[df_ast_try.keys().isin(df_minutos_cm.keys())]


In [13]:
'''
Creamos un dataframe con la media de los porcentajes de pases acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_cmp_try= df_cm.groupby('Player')['Passes|Cmp%'].mean().sort_values(ascending=False)
df_cmp= df_cmp_try[df_cmp_try.keys().isin(df_minutos_cm.keys())]
df_cmp= df_cmp/df_cmp.sum()



In [14]:
'''
Creamos un dataframe con los goles anotados por jugador.
'''
df_gls_try= df_cm.groupby('Player')['Performance|Gls'].sum().sort_values(ascending=False)
df_gls = df_gls_try[df_gls_try.keys().isin(df_minutos_cm.keys())]



In [15]:
'''
Creamos un dataframe con la media de los porcentajes de regates acertados por partido.
Posteriormente los ponderamos entre el total.
'''
df_drbls_try= df_cm.groupby('Player')['Dribbles|Succ_x'].sum().sort_values(ascending=False)
df_drbls_succ = df_drbls_try[df_drbls_try.keys().isin(df_minutos_cm.keys())]
df_drbls_succ = df_drbls_succ/(df_drbls_succ.sum())


In [16]:
#Convertimos las series a dataframe
df_recov=df_recov.to_frame()
df_cmp = df_cmp.to_frame()
df_ast = df_ast.to_frame()
df_gls = df_gls.to_frame()
df_drbls_succ = df_drbls_succ.to_frame()


In [17]:
'''
Unimos todos los dataframes por la columna Player
'''
df_cm_final = df_recov.merge(df_cmp, how= 'inner', on=['Player'])
df_cm_final = df_cm_final.merge(df_ast, how='inner', on=['Player'])
df_cm_final = df_cm_final.merge(df_gls, how='inner', on=['Player'])
df_cm_final =df_cm_final.merge(df_drbls_succ, how='inner', on=['Player'])


In [18]:
#añadimos la columna con los equipos de cada jugador al dataframe final
df_teams= df[['Player','Team']]
df_cm_final=df_teams.merge(df_cm_final, how='inner', on='Player').drop_duplicates()


In [19]:
#establecemos la Puntuación Final
df_cm_final['Puntuación Final'] = (df_cm_final.iloc[:,2] + (df_cm_final.iloc[:,3]*2) + (df_cm_final.iloc[:,4]*0.01) + (df_cm_final.iloc[:,5]*0.01) + (df_cm_final.iloc[:,6]))
df_cm = df_cm_final.sort_values(by='Puntuación Final', ascending=False) 

In [20]:
#Cambiamos el nombre de las columnas
df_cm.columns = ['Jugador', 'Equipo', 'Recuperaciones', 'Porcentaje de pases acertados', 'Asistencias', 'Goles', 'Regates Acertados', 'Puntuación Final']
df_cm = df_cm.sort_values(by='Puntuación Final', ascending=False) 
df_cm

,Jugador,Equipo,Recuperaciones,Porcentaje de pases acertados,Asistencias,Goles,Regates Acertados,Puntuación Final
128,Luka Modrić,RealM,0.031158,0.041747,1.0,2.0,0.118483,0.263136
104,Carlos Soler,Valencia,0.043535,0.036850,4.0,5.0,0.028436,0.235671
143,Mikel Merino,RealS,0.044885,0.036231,2.0,1.0,0.056872,0.204219
303,David Silva,RealS,0.036878,0.040219,2.0,1.0,0.056872,0.204189
314,Frenkie de Jong,Barcelona,0.040039,0.042215,1.0,0.0,0.066351,0.200819
227,Renato Tapia,Celta,0.054526,0.038759,0.0,0.0,0.066351,0.198395
201,Denis Suárez,Celta,0.032759,0.037330,2.0,0.0,0.066351,0.193769
14,Guido Rodríguez,Betis,0.055204,0.040566,0.0,1.0,0.047393,0.193730
281,Daniel Parejo,Villarreal,0.050000,0.041761,1.0,1.0,0.037915,0.191437
213,Nolito,Celta,0.028449,0.034229,1.0,3.0,0.052133,0.189039


In [21]:
#Exportar a csv
df_cm.to_csv('/Users/alexcar934/Desktop/EDA_bien/src/data/processed/csv_principales/5_Mediocentros.csv')